# 실전크롤링 - 1. 네이버 증권 뉴스
- 2024/04/30 네이버 증권 주요뉴스의 [제목, 상세페이지링크, 내용, 언론사, 날짜]를 엑셀에 저장하기
- 정적 크롤링 기초
- 마지막 페이지 찾기

In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/news/mainnews.naver'


response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

## 1) 첫번째 페이지 크롤링

In [ ]:
#1단계 : 첫번째 기사 가져오기
#2단계 : 첫번째 페이지 20개 가져오기
tags = soup.select('.block1')
for tag in tags:
    title = tag.select_one('.articleSubject > a').text #텍스트만 추출하는 법?
    link =  'https://finance.naver.com'  + tag.select_one('.articleSubject > a').attrs['href'] #링크 추출하는 법? > 전체로 추출
    summary = tag.select_one('.articleSummary').contents[0].strip() #첫번째 텍스트만 추출하는 법? /t 제외하기
    company = tag.select_one('.press').text.strip() #공백제거하기기
    time = tag.select_one('.wdate').text
    print('-------------', title, link, summary, company, time)
    

## 2) 첫번째 ~ 마지막페이지 크롤링

### a. 나의풀이

In [ ]:
#3단계 : 첫번째 ~ 마지막페이지까지 가져오기
# 나의 풀이
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://finance.naver.com/news/mainnews.naver'

response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

#마지막페이지 찾기
last_page = soup.select_one('.pgRR > a').attrs['href']
pages = int(last_page[-1])

#페이지마다 크롤링
output = pd.DataFrame(columns =['title','link','summary','company','time'])
page=1
i = 0
for page in range(pages+1):
    page_link = url + '?&page=' + str(page)

    response = requests.get(page_link)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    tags = soup.select('.block1')
    for tag in tags:
        title = tag.select_one('.articleSubject > a').text #텍스트만 추출하는 법?
        link =  'https://finance.naver.com'  + tag.select_one('.articleSubject > a').attrs['href'] #링크 추출하는 법? > 전체로 추출
        summary = tag.select_one('.articleSummary').contents[0].strip() #첫번째 텍스트만 추출하는 법? /t 제외하기
        company = tag.select_one('.press').text.strip() #공백제거하기
        time = tag.select_one('.wdate').text
        
        output.loc[i,'title'] = title
        output.loc[i,'link'] = link
        output.loc[i,'summary'] = summary
        output.loc[i,'company'] = company
        output.loc[i,'time'] = time
        i+=1
        
output.to_excel(f'C:/Users/qawse/세희/IT공부/크롤링_inflearn/output.xlsx', index=None)
        

### b. 강의풀이

In [ ]:
#3단계 : 첫번째 ~ 마지막페이지까지 가져오기
#> 맨 뒤 버튼의 유무를 확인하여 반복 멈추기 **
# 리스트에 2차원 객체로 담아 한번에 df로 변경하기 **
import requests
from bs4 import BeautifulSoup
import pandas as pd

output = []
for i in range(1, 1000):
    url = 'https://finance.naver.com/news/mainnews.naver?&page='+str(i)
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    tags = soup.select('.block1')
    for tag in tags:
        title = tag.select_one('.articleSubject > a').text #텍스트만 추출하는 법?
        link =  'https://finance.naver.com'  + tag.select_one('.articleSubject > a').attrs['href'] #링크 추출하는 법? > 전체로 추출
        summary = tag.select_one('.articleSummary').contents[0].strip() #첫번째 텍스트만 추출하는 법? /t 제외하기
        company = tag.select_one('.press').text.strip() #공백제거하기
        time = tag.select_one('.wdate').text
        # print(title, link, summary, company, time)

        output.append([title, link, summary, company, time])

    if soup.select_one('.pgRR') == None: #없으면 None 객체가 반환된다.
        break

df = pd.DataFrame(output, columns = ['title','link','summary','company','time']) 
df.to_excel(f'C:/Users/qawse/세희/IT공부/크롤링_inflearn/naver_finance_crawling.xlsx', index=None)
